In [17]:
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4()

### Creating Julep Client with the API Key

Get you API key from [here](https://dashboard.julep.ai/)

In [18]:
from julep import Client
import os
from dotenv import load_dotenv

load_dotenv()

JULEP_API_KEY = os.environ["JULEP_API_KEY"]

# Create a Julep client
client = Client(api_key=JULEP_API_KEY, environment="production")

In [28]:
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name="Julep Email Assistant",
    about=(
        "You are an agent that handles emails for julep users."
        + " Julep is a platform for creating kick-ass AI agents."
    ),
    model="gpt-4o",
)

In [29]:
import yaml

MAILGUN_PASSWORD = os.environ['MAILGUN_PASSWORD']

task_def = yaml.safe_load(f"""
# yaml-language-server: $schema=https://raw.githubusercontent.com/julep-ai/julep/refs/heads/dev/schemas/create_task_request.json
name: Julep Email Assistant
description: A Julep agent that can send emails and search the documentation.

########################################################
####################### INPUT SCHEMA ###################
########################################################
input_schema:
  type: object
  properties:
    from:
      type: string
    to:
      type: string
    subject:
      type: string
    body:
      type: string

                          
########################################################
####################### TOOLS ##########################
########################################################

# Define the tools that the task will use in this workflow
tools:
- name: send_email
  type: integration
  integration:
    provider: email
    setup:
      host: smtp.mailgun.org
      password: {MAILGUN_PASSWORD}
      port: 587
      user: postmaster@sandbox8fb6fbc34368497b877d285ae92db89e.mailgun.org

- name: search_docs
  type: system
  system:
    resource: agent
    subresource: doc
    operation: search
  
########################################################
####################### MAIN WORKFLOW ##################
########################################################

main:
- prompt: |-
    $ f'''You are {{ agent.name }}. {{ agent.about }}

    you have to email to email address {{ _['to'] }} :
    with the following body and subject
    ------
      Subject: {{ _.subject }}

      {{ _.body }}
    ------

    Based on the above info, craft an email body to respond with as a json object.
    The json object must have `subject` and `body` fields.'''
  response_format:
    type: json_object

  unwrap: true


# Step 3: Extract the email
- evaluate:
    subject: $ extract_json(_)['subject']
    body: $ extract_json(_)['body']

# Step 4: Send the email
- tool: send_email
  arguments:
    body: $ _.body
    from: postmaster@sandbox8fb6fbc34368497b877d285ae92db89e.mailgun.org
    subject: $  _.subject
    to: $ steps[0].input['to']
""")

In [30]:
task = client.tasks.create_or_update(
    agent_id=AGENT_UUID,
    task_id=TASK_UUID,
    **task_def,
)

In [31]:
execution = client.executions.create(
    task_id=task.id,
    input={"from": "postmaster@sandbox8fb6fbc34368497b877d285ae92db89e.mailgun.org", "to": "akshaw.ak4@gmail.com", "subject": "what's up", "body": "sup"},
)

In [32]:
client.executions.get(execution.id)

Execution(id='0680170f-d2b2-78d5-8000-846f9ea05670', created_at=datetime.datetime(2025, 4, 17, 21, 22, 5, 173540, tzinfo=datetime.timezone.utc), input={'to': 'akshaw.ak4@gmail.com', 'body': 'sup', 'from': 'postmaster@sandbox8fb6fbc34368497b877d285ae92db89e.mailgun.org', 'subject': "what's up"}, status='starting', task_id='5aceaf4d-bda9-4c11-adb7-c0b10b9d0b5b', updated_at=datetime.datetime(2025, 4, 17, 21, 22, 5, 434078, tzinfo=datetime.timezone.utc), error=None, metadata={}, output={'to': 'akshaw.ak4@gmail.com', 'body': 'sup', 'from': 'postmaster@sandbox8fb6fbc34368497b877d285ae92db89e.mailgun.org', 'subject': "what's up"}, transition_count=1)

In [34]:
import time
execution = client.executions.get(execution.id)

while execution.status != "succeeded":
    time.sleep(5)
    execution = client.executions.get(execution.id)
    print("Execution status: ", execution.status)
    print("-"*50)

Execution status:  running
--------------------------------------------------


KeyboardInterrupt: 

In [33]:
execution_transitions = client.executions.transitions.list(
    execution_id=execution.id).items

for transition in reversed(execution_transitions):
    print("Type: ", transition.type)
    print("output: ", transition.output)
    print("-" * 100)

Type:  init
output:  {'to': 'akshaw.ak4@gmail.com', 'body': 'sup', 'from': 'postmaster@sandbox8fb6fbc34368497b877d285ae92db89e.mailgun.org', 'subject': "what's up"}
----------------------------------------------------------------------------------------------------
Type:  step
output:  ```json
{
  "subject": "what's up",
  "body": "sup"
}
```
----------------------------------------------------------------------------------------------------
Type:  step
output:  {'body': 'sup', 'subject': "what's up"}
----------------------------------------------------------------------------------------------------
